In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import geopandas as gp

In [2]:
list_xls_file = []
floder_path = '104年火災事件資料/'
for i in os.listdir(floder_path):
    list_xls_file.append(floder_path+i)

In [11]:
df_raw = pd.DataFrame()
for xls in list_xls_file:
    df = pd.read_excel(
                  xls, #list_xls_file[3],
                  skiprows=[0] #這份文件的 row1 不是 index，所以要跳過
    )
    print('{} 案件數量={}, '.format(df['縣市'].unique(), df.index.size), end='')
    df_raw = pd.concat([df_raw, df])

['高雄市' '臺南市'] 案件數量=6, ['新北市'] 案件數量=130, ['臺北市'] 案件數量=103, ['桃園市'] 案件數量=296, ['臺中市'] 案件數量=146, ['臺南市'] 案件數量=103, ['高雄市'] 案件數量=61, ['宜蘭縣'] 案件數量=69, ['新竹縣'] 案件數量=44, ['苗栗縣'] 案件數量=36, ['彰化縣'] 案件數量=88, ['南投縣'] 案件數量=108, ['雲林縣'] 案件數量=119, ['嘉義縣'] 案件數量=40, ['屏東縣'] 案件數量=83, ['臺東縣'] 案件數量=56, ['花蓮縣'] 案件數量=25, ['澎湖縣'] 案件數量=27, ['基隆市'] 案件數量=39, ['新竹市'] 案件數量=39, ['嘉義市'] 案件數量=24, ['金門縣'] 案件數量=11, ['連江縣'] 案件數量=4, ['臺中港'] 案件數量=15, 

In [10]:
# 火災事件關聯分組
dict_fire_event = {
    '自殺':['自殺','自焚'],
    '蠟燭':['蠟燭','燈燭'],
    '縱火':['縱火','疑似縱火','人為縱火'],
    '瓦斯':['瓦斯','瓦斯外洩','瓦斯漏氣或爆炸','洩漏液化石油氣','洩漏液化瓦斯','瓦斯漏氣爆炸','瓦斯灌充填打火機不慎'],
    '機械設備':['機械設備','機械故障'],
    '電氣因素':['電氣因素','電氣因素(不含車輛)'],
    '乾燥作業':['乾燥作業','烘乾機作業不慎','烤火'],
    '敬神祭祖':['敬神祭祖','敬神、祭祖','敬神','祭祖'],
    '爐火烹調':['爐火烹調','爐火不慎','爐火烹調油鍋起火','爐火烹調乾燒'],
    '車輛因素':['車輛因素','車輛電氣因素','車輛機械因素','引擎排煙管高溫','車輛人為疏忽','車輛電氣系統','車輛燃料系統',],
    '燃燒廢棄物':['燃燒廢棄物','燃燒雜草、垃圾','自清','燒廢料','燃燒木材不慎','整地','引燃汽油','引燃枯草'],
    '燃放爆竹煙火':['燃放爆竹','燃放爆竹煙火','施放爆竹煙火'],
    '易燃液體接觸火源':['易燃液體接觸火源','易燃品自燃','化學物品','柴油洩漏接觸熱源','絕緣油接觸高溫','易燃液體'],
    '其他或不明':['其他或不明','其他','其它','原因不明','不明']
}

In [6]:
print("原先所有類別數量 = {}".format(len(df_raw['起火原因'].unique())))

for key in dict_fire_event:
    df_raw.replace(
        to_replace = dict_fire_event[key],
        value = key, inplace=True)

df = df_raw.groupby(by=['起火原因'], squeeze=True).count()
df = df.iloc[:,0:1].sort_values(by=df.columns[0], ascending=False)
print("關聯分組類別數量 = {}".format(len(df.index.unique())))
print(df.index.values)

原先所有類別數量 = 62
關聯分組類別數量 = 25
['電氣因素' '縱火' '其他或不明' '菸蒂' '遺留火種' '爐火烹調' '車輛因素' '施工不慎' '燃燒廢棄物' '敬神祭祖' '瓦斯'
 '機械設備' '燃放爆竹煙火' '自殺' '蠟燭' '交通事故' '易燃液體接觸火源' '玩火' '敬神掃墓祭祖' '乾燥作業' '汙泥過熱'
 '除蜂不慎' '電暖器引燃可燃物' '天然災害' '點蚊香不慎引火']


In [8]:
df

,縣市
起火原因,
電氣因素,576
縱火,270
其他或不明,164
菸蒂,148
遺留火種,104
爐火烹調,72
車輛因素,46
施工不慎,37
燃燒廢棄物,31
